### Requirements

#### Python packages
- [cython](https://pypi.org/project/Cython/)
- [MDAnalysis](https://pypi.org/project/MDAnalysis/)
- [periodictable](https://pypi.org/project/periodictable/)


In [ ]:
import numpy as np
import MDAnalysis as mda
import periodictable
import nsc

### Download water simulation

In [4]:
%%capture
%%bash
wget https://www.dropbox.com/s/au6k0m84kggq057/water.tar
tar xf water.tar

### Create system and extract coordinates

In [14]:
%%capture
u = mda.Universe("water/npt.gro", "water/nvt.trr")

`u` is a `MDAnalysis.Universe` object. It contains 512 TIP4P water molecules. Each TIP4P water molecule is composed of one oxygen, two hydrogens, and two lone-pair electrons.

In [41]:
print('number of residues =', len(u.residues))
print('Atoms in the first water molecule', u.atoms[0:4])

number of residues = 512
Atoms in the first water molecule <AtomGroup [<Atom 1: OW of type O of resname SOL, resid 1 and segid SYSTEM>, <Atom 2: HW1 of type H of resname SOL, resid 1 and segid SYSTEM>, <Atom 3: HW2 of type H of resname SOL, resid 1 and segid SYSTEM>, <Atom 4: LP1 of type L of resname SOL, resid 1 and segid SYSTEM>]>


Select only the hydrogens and oxygens. There should be three atoms per water molecule

In [33]:
sel = u.select_atoms('type H or type O')
print('number of atoms per water molecule = ', len(sel)/len(u.residues))

number of atoms per water molecule =  3.0


Obtain the bound coherent neutron scattering lengths for our selection.

In [38]:
# the detailed code. See the end of this cell for the one-liner version.
b_c = list()
for atom in sel:
    element_name = atom.type
    element = getattr(periodictable, element_name)
    b_c.append(element.neutron.b_c)
b_c = np.asarray(b_c)
# The one-liner version
#b_c = np.asarray([getattr(periodictable, at.type).neutron.b_c  for at in sel])

In [39]:
print(b_c)  # units in fm

[ 5.805  -3.7409 -3.7409 ...  5.805  -3.7409 -3.7409]


We investigate the trajectory.

In [52]:
print('number of conformations =', u.trajectory.n_frames)

number of conformations = 5001


We will calculate the structure factor for 1000 conformations, out of the 5001. We extract one conformation every 5. We only care about our hydrogen + oxygen selection `sel`.

In [59]:
xyz = np.asarray([sel.positions for _ in u.trajectory[0:-1:5]])  # from beginning (0) to end (-1) every 5

In [61]:
print(xyz.shape)  #  1000 conformations, 1536 atoms per conformation, 3 coordinates (x, y, z) per atoms

(1000, 1536, 3)
